In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the DeepSolar Data and Fields List

In [2]:
solar=pd.read_csv('../../deepsolar_tract.csv',encoding = "ISO-8859-1")
solar.head()
solar_fields=pd.read_csv('../deepsolar fields.csv')
solar_fields.head()

,Field,Description,Unit,Data Type,Formula,Possible Values,Observed Max,Observed Min,Theoretical Min,Theoretical Max,Relevant Feature
0,Unnamed: 0,Index,NaN,Numeric,NaN,NaN,72537.000,0,NaN,NaN,0
1,tile_count,total number of tiles in census tract,NaN,Numeric,NaN,NaN,4468.000,0,0.0,NaN,0
2,solar_system_count,Total number of solar systems in census tract,NaN,Numeric,NaN,NaN,1535.000,0,0.0,NaN,0
3,total_panel_area,NaN,NaN,Numeric,NaN,NaN,592031.075,0,0.0,NaN,0
4,fips,FIPS identifier for the census tract,NaN,String,NaN,NaN,NaN,NaN,NaN,NaN,0


In [3]:
solar

,Unnamed: 0,tile_count,solar_system_count,total_panel_area,fips,average_household_income,county,education_bachelor,education_college,education_doctoral,...,incentive_count_nonresidential,incentive_residential_state_level,incentive_nonresidential_state_level,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate
0,0,0.0,0.0,0.000000,27145011200,70352.789869,Stearns County,569,1690,13,...,39,11,13,34,0,0,25,12,0,9.46
1,1,25.0,21.0,1133.436461,27145011301,61727.085202,Stearns County,674,1434,108,...,39,11,13,34,0,0,25,12,0,9.46
2,2,3.0,3.0,64.505776,27145011302,71496.886583,Stearns County,854,1459,31,...,39,11,13,34,0,0,25,12,0,9.46
3,3,0.0,0.0,0.000000,27145011304,86840.152755,Stearns County,640,1116,68,...,39,11,13,34,0,0,25,12,0,9.46
4,4,5.0,5.0,164.583303,27145011400,89135.315597,Stearns County,654,1314,15,...,39,11,13,34,0,0,25,12,0,9.46
5,5,0.0,0.0,0.000000,27145011500,62225.903614,Stearns County,522,1395,24,...,39,11,13,34,0,0,25,12,0,9.46
6,6,2.0,2.0,25.299013,27145011600,41068.936170,Stearns County,49,278,32,...,39,11,13,34,0,0,25,12,0,9.46
7,7,0.0,0.0,0.000000,27145010500,74073.833671,Stearns County,242,867,10,...,39,11,13,34,0,0,25,12,0,9.46
8,8,0.0,0.0,0.000000,27145011100,69412.192435,Stearns County,527,1665,6,...,39,11,13,34,0,0,25,12,0,9.46
9,9,11.0,10.0,415.365350,27145010102,82502.407069,Stearns County,1582,1949,6,...,39,11,13,34,0,0,25,12,0,9.46


# Load/Test PySpark

In [4]:
from pyspark import SparkContext
sc = SparkContext()

In [5]:
import numpy as np

TOTAL = 100
dots = sc.parallelize([2.0 * np.random.random(2) - 1.0 for i in range(TOTAL)]).cache()
print("Number of random points:", dots.count())

stats = dots.stats()
print('Mean:', stats.mean())
print('stdev:', stats.stdev())

Number of random points: 100
Mean: [ 0.08481536 -0.00102725]
stdev: [ 0.58715975  0.58795094]


# Process DeepSolar Data

## Create Training/Dev Sets

In [12]:
features=solar_fields.loc[(solar_fields['Relevant Feature']==1)]['Field'].tolist()
all_variables=features+['number_of_solar_system_per_household']

#Set infinity and blank spaces to NaN in independent variables, set infinite to 0 in dependent variable
solar2=solar[features].replace([np.inf,' '],np.nan)
solar2['number_of_solar_system_per_household']=solar['number_of_solar_system_per_household'].replace([np.inf,np.nan],0)

#Create binary version of number_of_solar_system_per_household for RF classifier

solar2['solar_flag']=solar2['number_of_solar_system_per_household'].apply(lambda x: int(x>0))


#create dummy variables for state
solar2=pd.get_dummies(solar2,columns=['state'])

#create binary version of vote dem win variables

solar2['voting_2016_dem_win']=solar2['voting_2016_dem_win'].apply(lambda x: int(x))
solar2['voting_2012_dem_win']=solar2['voting_2012_dem_win'].apply(lambda x: int(x))


#designate independent variable frame
independent_vars=solar2.loc[:,~solar2.columns.isin(['number_of_solar_system_per_household','solar_flag'])]

#create training and test data
shuffle = np.random.permutation(np.arange(independent_vars.shape[0]))
split_size=int(shuffle.shape[0]*0.8)

X,y=independent_vars.values[shuffle],solar2['solar_flag'].values[shuffle]
X_train,y_train=X[0:split_size],y[0:split_size]
X_dev,y_dev=X[split_size:],y[split_size:]
print('training data shape: ',X_train.shape)
print('training labels shape: ',y_train.shape)
print('dev data shape: ',X_dev.shape)
print('dev labels shape: ',y_dev.shape)


training data shape:  (58029, 168)
training labels shape:  (58029,)
dev data shape:  (14508, 168)
dev labels shape:  (14508,)


## Train the Model/Make Predictions

In [18]:
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

In [20]:
#convert training data to LabeledPoint data type
data=[]
for i in range(X_train.shape[0]):
    data.append(LabeledPoint(y_train[i],X_train[i]))

## Train the Model

In [22]:
model = RandomForest.trainClassifier(sc.parallelize(data), 2, {}, 3, seed=42)

## Obtain Dev Set Accuracy

In [24]:
dev_set=[]
for i in range(X_dev.shape[0]):
    dev_set.append(list(X_dev[i]))

In [27]:
preds=model.predict(sc.parallelize(dev_set)).collect()

In [29]:
preds_array=np.array(preds)
preds_array

array([ 1.,  1.,  0., ...,  1.,  1.,  1.])

In [30]:
(preds_array==y_dev).mean()

0.75372208436724564